In [2]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import os, time, uuid

In [3]:
# Putting in key, endpoint and id values
ENDPOINT = "https://yoyoyo.cognitiveservices.azure.com/"
ENDPOINT2 = "https://yoyoyo-prediction.cognitiveservices.azure.com/"
training_key = "25cfe69107754f55ac33b305a527fe61"
prediction_key = "e80894bfa56249c0b5238e2d5b936c70"
prediction_resource_id = "/subscriptions/18ae480a-58ae-4be3-baa4-318cc8b683fb/resourceGroups/test-group/providers/Microsoft.CognitiveServices/accounts/yoyoyo-Prediction"

In [4]:
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT2, prediction_credentials)


In [ ]:
publish_iteration_name = "detectModel"

# Finding the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")

# Creating a new project
print ("Creating project NOW NOW NOW")

# Using uuid to avoid project name collisions.
project = trainer.create_project(str(uuid.uuid4()), domain_id=obj_detection_domain.id)

In [20]:
# Adding in the defect1 tag
defect1_tag = trainer.create_tag(project.id, "defect1")

In [5]:
import pandas as pd

images = pd.read_csv('Train_DefectBoxes_PrithviAI.csv')

In [ ]:
images.dropna(inplace=True)
images.X = images.X - images.W/2
images.Y = images.Y - images.H/2

defect1_image_regions = images.set_index("  image_id").T.to_dict('list')

In [ ]:
images.Y

In [ ]:
base_image_location = "New_folder/Images_Unzipped/New_Images4"

# Going through the data above and creating the images
print ("Adding images")
tagged_images_with_regions = []

for file_name in defect1_image_regions.keys():
    x,y,w,h = defect1_image_regions[file_name]
    regions = [Region(tag_id=defect1_tag.id, left=x,top=y,width=w,height=h) ]

    try:
        img_file = os.path.join(base_image_location, "new_" + file_name)
        with open(img_file, mode="rb") as image_contents:
            tagged_images_with_regions.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), regions=regions))
    except:
        pass

print(tagged_images_with_regions)

In [ ]:
print(len(tagged_images_with_regions))

In [ ]:
i = 0
while i <= len(tagged_images_with_regions):
    if i + 5 > len(tagged_images_with_regions):
        tagged_images_with_regions_upload = tagged_images_with_regions[i:len(tagged_images_with_regions)]
    else:
        tagged_images_with_regions_upload = tagged_images_with_regions[i:i+5]
    upload_result = trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=tagged_images_with_regions_upload))
    i += 5

    if not upload_result.is_batch_successful:
        print("Image batch upload failed.")
        for image in upload_result.images:
            print("Image status: ", image.status)
        exit(-1)

In [ ]:
print ("Training NOW")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print("Training status: " + iteration.status)
    time.sleep(1)

In [ ]:
# The iteration is now trained. Publish it to the project endpoint
trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)
print ("Finally, Done")

In [ ]:
# Now there is a trained endpoint that can be used to make a prediction
ENDPOINT2 = "https://yoyoyo-prediction.cognitiveservices.azure.com/"
predictor = CustomVisionPredictionClient(ENDPOINT2, prediction_credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})

# Open the sample image and get back the prediction results.
with open(os.path.join ("test_image.png"), mode="rb") as test_data:
    results = predictor.detect_image(project.id, publish_iteration_name, test_data)

# Display the results.    
for prediction in results.predictions:
    print("\t" + prediction.tag_name + ": {0:.2f}% bbox.left = {1:.2f}, bbox.top = {2:.2f}, bbox.width = {3:.2f}, bbox.height = {4:.2f}".format(prediction.probability * 100, prediction.bounding_box.left, prediction.bounding_box.top, prediction.bounding_box.width, prediction.bounding_box.height))
